In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [2]:
from case_code import CodeDownloader, get_metadata, CodeLoader, transform, Case

In [3]:
# Download all case code
downloader = CodeDownloader()
downloaded_count = downloader.download()
print(f"{'Total files downloaded:':<24} {downloaded_count}")

metadata = get_metadata()
print(f"{'Total cases in metadata:':<24} {len(metadata)}")

Total files downloaded:  125
Total cases in metadata: 110


In [4]:
from utils.model_selector import get_chat_model
import pandas as pd

# Get the chat model
model = get_chat_model("openai")

# Transform the downloaded code to case
loader = CodeLoader()
skipped = await transform(model=model, loader=loader, total=downloaded_count)

data = [(key.value, len(value)) for key, value in skipped.items()]
df = pd.DataFrame(data, columns=["Error", "count"])
print(df)

Processing: 100%|██████████| 125/125 [04:31<00:00,  2.17s/it]

            Error  count
0  Case Not Found     57
1     Parse Error      1


In [5]:
from case_code.code_transformer import TransformError

print(f"Parsed error cases: {skipped[TransformError.ParseError]}")

Parsed error cases: ['zircuit-ethena/usdt-case.ts']


In [6]:
from case import get_retriever

retriever = get_retriever()

In [7]:
retriever.invoke("Stake ETH with Lido and deposit to Eigenpie")

[Document(metadata={'case_id': 'eigenpie_steth', 'seq_num': 2, 'source': '/Users/clement/Developer/ai/tx-generator/data/case_gpt-4o.jsonl', 'steps': "[{'description': 'ETH to Lido', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': '{user_input}', 'function_name': 'submit', 'input_args': ['ReferalAccount']}, {'description': 'Approve stETH to Eigenpie', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': '0', 'function_name': 'approve', 'input_args': ['0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', '{user_input}']}, {'description': 'stETH to Eigenpie', 'to': '0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', 'value': '0', 'function_name': 'depositAsset', 'input_args': ['0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', '{user_input}', '{user_input}', 'ReferalAccount']}]", 'total_steps': 3}, page_content='Staking ETH with Lido and then depositing stETH to Eigenpie.'),
 Document(metadata={'case_id': 'stake_eth', 'seq_num': 47, 'source': '/Users/clement/Developer/ai/tx-generator/da

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

template = """Generate a structure output based only on the following context:
{context}
Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

retriever = get_retriever()


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(Case)
)

In [9]:
question = "Stake 0.01 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"case_id": "eigenpie_steth", "description": "Stake 0.01 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "ETH to Lido", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0.01", "function_name": "submit", "input_args": ["ReferalAccount"]}, {"description": "Approve stETH to Eigenpie", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0", "function_name": "approve", "input_args": ["0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "{user_input}"]}, {"description": "stETH to Eigenpie", "to": "0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "value": "0", "function_name": "depositAsset", "input_args": ["0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "{user_input}", "{user_input}", "ReferalAccount"]}]}


In [10]:
question = "Stake 3 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"case_id": "stake_eth_eigenpie", "description": "Stake 3 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "ETH to Lido", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "3", "function_name": "submit", "input_args": ["ReferalAccount"]}, {"description": "Approve stETH to Eigenpie", "to": "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "value": "0", "function_name": "approve", "input_args": ["0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "{user_input}"]}, {"description": "stETH to Eigenpie", "to": "0x24db6717dB1C75B9Db6eA47164D8730B63875dB7", "value": "0", "function_name": "depositAsset", "input_args": ["0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84", "{user_input}", "{user_input}", "ReferalAccount"]}]}


In [11]:
question = "Swap 100 USDT for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"case_id": "swap_usdt_usdc_uniswap", "description": "Swap 100 USDT for USDC on Uniswap", "total_steps": 2, "steps": [{"description": "Approve USDT to Uniswap", "to": "0xdAC17F958D2ee523a2206206994597C13D831ec7", "value": "0", "function_name": "approve", "input_args": ["0xE592427A0AEce92De3Edee1F18E0157C05861564", "100"]}, {"description": "Swap USDT to USDC", "to": "0xE592427A0AEce92De3Edee1F18E0157C05861564", "value": "0", "function_name": "exactInputSingle", "input_args": ["0xdAC17F958D2ee523a2206206994597C13D831ec7", "0xA0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "3000", "{recipient}", "{deadline}", "100", "{usdcAmount}", "0"]}]}


In [12]:
question = "Swap 0.001 ETH for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"case_id": "uniswap_eth_usdc_swap", "description": "Swap 0.001 ETH for USDC on Uniswap", "total_steps": 1, "steps": [{"description": "Swap 0.001 ETH for USDC", "to": "UniswapRouterAddress", "value": "0.001", "function_name": "swapExactETHForTokens", "input_args": ["{amountOutMin}", "[{WETHAddress},{USDCAddress}]", "{recipient}", "{deadline}"]}]}


In [13]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into a Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
print(result.json())

{"case_id": "yield_farming_eth_dai", "description": "Yield Farming with Multiple Protocols", "total_steps": 4, "steps": [{"description": "Deposit ETH into AAVE to earn interest", "to": "0x3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3", "value": "{user_input}", "function_name": "depositETH", "input_args": ["0x3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3", "{user_input}", "0"]}, {"description": "Use aToken (aETH) as collateral to borrow DAI", "to": "0x3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3", "value": "0", "function_name": "borrowDAI", "input_args": ["0x3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3", "{user_input}", "2", "0"]}, {"description": "Deposit the borrowed DAI into Yearn Finance", "to": "0x3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3", "value": "0", "function_name": "depositDAI", "input_args": ["0xYearnVaultAddress", "{user_input}", "0"]}, {"description": "Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees", "to": "0x3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3d4E3fD3", "value": "0", 